<a href="https://colab.research.google.com/github/DeepArt84/Artability/blob/main/Artability_speechRecognition_wav2vec_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Creating Captions with Wav2Vec

---

## Setup

---

In [ ]:
!pip -q install transformers
!pip -q install youtube_dl

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 48.7 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 1.9 MB 5.2 MB/s 


In [ ]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from IPython.display import Audio

import moviepy.editor as mp
import torch
import librosa
import os

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2539520/45929032 bytes (5.5%)5799936/45929032 bytes (12.6%)9052160/45929032 bytes (19.7%)12066816/45929032 bytes (26.3%)15310848/45929032 bytes (33.3%)18489344/45929032 bytes (40.3%)21741568/45929032 bytes (47.3%)25133056/45929032 bytes (54.7%)28467200/45929032 bytes (62.0%)31293440/45929032 bytes (68.1%)34734080/45929032 bytes (75.6%)38100992/45929032 bytes (83.0%)

## Get Clip

---

In [ ]:

clip = "/content/olayan.mp4"

# Substitue below for start/end seconds
start = 0
end = 10

In [ ]:
# Download the clip as mp4 & rename it for usability
os.system('youtube-dl {} --recode-video mp4'.format(clip))
os.system('mv *.mp4 olayan.mp4')

256

## Model and tokenizer

---

Load the Wav2Vec model from 🤗 Transformers. See [here](https://huggingface.co/transformers/model_doc/wav2vec2.html) for the models documentation.

In [ ]:
# Load Wav2Vec from huggingface
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Extract Audio

---

First we'll extract the audio in mp3 format from the clip, as the Wav2Vec models expects audio input. We do this in subclips of 10 second length to save some memory lateron. 

In [ ]:
clip = mp.VideoFileClip("olayan.mp4")
end = min(clip.duration, end)

# Save the paths for later
clip_paths = []

# Extract Audio-only from mp4
for i in range(start, int(end), 10):
  sub_end = min(i+10, end)
  sub_clip = clip.subclip(i,sub_end)

  sub_clip.audio.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

[MoviePy] Writing audio in audio_0.mp3


100%|██████████| 121/121 [00:00<00:00, 979.08it/s] 

[MoviePy] Done.


In [ ]:
# Play Audio 
Audio(clip_paths[0])

## Transcribe Audio

---

The last step is turning the Audio into text! The Wav2Vec model does most of the job here for us. We do each 10-second clip one-by-one to save memory.

In [ ]:
cc = ""

for path in clip_paths:
    # Load the audio with the librosa library
    input_audio, _ = librosa.load(path, 
                                sr=16000)

    # Tokenize the audio
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    # Feed it through Wav2Vec & choose the most probable tokens
    with torch.no_grad():
      logits = model(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)

    # Decode & add to our caption string
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    cc += transcription + " "

In [ ]:
# Here's your caption!
# Note that there may be mistakes especially if the audio is noisy or there are uncommon words
# If you picked the default video and change start to 0, you will see that the model gets confused by the word "Anakin"
print(cc)

fantasy colors with a dream in the space
